## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys

# Import API key
sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Carnarvon,AU,-24.8667,113.6333,78.80,100,0,10.36,clear sky
1,1,Ushuaia,AR,-54.8000,-68.3000,66.20,45,40,4.61,light rain
2,2,Hermanus,ZA,-34.4187,19.2345,59.00,88,1,10.69,clear sky
3,3,Lebu,CL,-37.6167,-73.6500,58.82,73,0,15.99,clear sky
4,4,Shestakovo,RU,56.4915,103.9600,-9.98,86,100,7.20,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Carnarvon,AU,-24.8667,113.6333,78.80,100,0,10.36,clear sky
5,5,Vaini,TO,-21.2000,-175.2000,86.00,79,40,11.50,scattered clouds
10,10,Cidreira,BR,-30.1811,-50.2056,76.30,83,13,21.74,few clouds
19,19,Hervey Bay,AU,-25.2986,152.8535,82.99,69,38,3.00,scattered clouds
21,21,Arraial Do Cabo,BR,-22.9661,-42.0278,76.32,86,0,21.99,clear sky
22,22,Kavieng,PG,-2.5744,150.7967,75.92,93,100,15.46,heavy intensity rain
23,23,Ambon,ID,-3.6954,128.1814,79.00,89,20,5.75,few clouds
24,24,Rikitea,PF,-23.1203,-134.9692,79.45,67,29,9.57,scattered clouds
26,26,Hithadhoo,MV,-0.6000,73.0833,81.28,78,98,14.90,overcast clouds
31,31,Saint-Philippe,RE,-21.3585,55.7679,78.80,83,0,5.75,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                195
City                   195
Country                194
Lat                    195
Lng                    195
Max Temp               195
Humidity               195
Cloudiness             195
Wind Speed             195
Current Description    195
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 4c. Check data again
clean_df.count()

City_ID                194
City                   194
Country                194
Lat                    194
Lng                    194
Max Temp               194
Humidity               194
Cloudiness             194
Wind Speed             194
Current Description    194
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Carnarvon,AU,78.80,clear sky,-24.8667,113.6333,
5,Vaini,TO,86.00,scattered clouds,-21.2000,-175.2000,
10,Cidreira,BR,76.30,few clouds,-30.1811,-50.2056,
19,Hervey Bay,AU,82.99,scattered clouds,-25.2986,152.8535,
21,Arraial Do Cabo,BR,76.32,clear sky,-22.9661,-42.0278,
22,Kavieng,PG,75.92,heavy intensity rain,-2.5744,150.7967,
23,Ambon,ID,79.00,few clouds,-3.6954,128.1814,
24,Rikitea,PF,79.45,scattered clouds,-23.1203,-134.9692,
26,Hithadhoo,MV,81.28,overcast clouds,-0.6000,73.0833,
31,Saint-Philippe,RE,78.80,light rain,-21.3585,55.7679,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# View output
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Carnarvon,AU,78.80,clear sky,-24.8667,113.6333,Hospitality Carnarvon
5,Vaini,TO,86.00,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
10,Cidreira,BR,76.30,few clouds,-30.1811,-50.2056,Hotel Castelo
19,Hervey Bay,AU,82.99,scattered clouds,-25.2986,152.8535,Shelly Bay Resort
21,Arraial Do Cabo,BR,76.32,clear sky,-22.9661,-42.0278,Pousada Porto Praia
22,Kavieng,PG,75.92,heavy intensity rain,-2.5744,150.7967,Nusa Island Retreat
23,Ambon,ID,79.00,few clouds,-3.6954,128.1814,Swiss-Belhotel Ambon
24,Rikitea,PF,79.45,scattered clouds,-23.1203,-134.9692,Pension Maro'i
26,Hithadhoo,MV,81.28,overcast clouds,-0.6000,73.0833,Scoop Guest House
31,Saint-Philippe,RE,78.80,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""


In [11]:
# Replace blank values for NaN
import numpy as np
hotel_df = hotel_df.replace(r'', np.NaN)
hotel_df.head(23)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Carnarvon,AU,78.80,clear sky,-24.8667,113.6333,Hospitality Carnarvon
5,Vaini,TO,86.00,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
10,Cidreira,BR,76.30,few clouds,-30.1811,-50.2056,Hotel Castelo
19,Hervey Bay,AU,82.99,scattered clouds,-25.2986,152.8535,Shelly Bay Resort
21,Arraial Do Cabo,BR,76.32,clear sky,-22.9661,-42.0278,Pousada Porto Praia
22,Kavieng,PG,75.92,heavy intensity rain,-2.5744,150.7967,Nusa Island Retreat
23,Ambon,ID,79.00,few clouds,-3.6954,128.1814,Swiss-Belhotel Ambon
24,Rikitea,PF,79.45,scattered clouds,-23.1203,-134.9692,Pension Maro'i
26,Hithadhoo,MV,81.28,overcast clouds,-0.6000,73.0833,Scoop Guest House
31,Saint-Philippe,RE,78.80,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""


In [12]:
# 7. Drop the rows where there is no Hotel Name. 
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])
clean_hotel_df.head(23)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Carnarvon,AU,78.80,clear sky,-24.8667,113.6333,Hospitality Carnarvon
5,Vaini,TO,86.00,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
10,Cidreira,BR,76.30,few clouds,-30.1811,-50.2056,Hotel Castelo
19,Hervey Bay,AU,82.99,scattered clouds,-25.2986,152.8535,Shelly Bay Resort
21,Arraial Do Cabo,BR,76.32,clear sky,-22.9661,-42.0278,Pousada Porto Praia
22,Kavieng,PG,75.92,heavy intensity rain,-2.5744,150.7967,Nusa Island Retreat
23,Ambon,ID,79.00,few clouds,-3.6954,128.1814,Swiss-Belhotel Ambon
24,Rikitea,PF,79.45,scattered clouds,-23.1203,-134.9692,Pension Maro'i
26,Hithadhoo,MV,81.28,overcast clouds,-0.6000,73.0833,Scoop Guest House
31,Saint-Philippe,RE,78.80,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""


In [13]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="WeatherPy_vacation_ID")

In [26]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [27]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))